# Implementation of the EBfilter by genomon

* EBrun (originally EBFilter) is an argparse wrapper passing command line arguments to run.py (is not needed for internal use)
* passed arguments:
    * targetMutationFile: the .vcf or .anno containing the mutations – needed --> mut_file
    * targetBamPath: path to the tumor bam file (+.bai) – needed --> tumor_bam
    * controlBamPathList: text list of path to PoN bam files (+ .bai) – needed --> pon_list
    * outputPath: clear  – needed --> output_path
    * -f option for anno or vcf – not needed --> will be inferred from .ext
    * thread_num: –not needed --> taken from config
    * -q option for quality threshold – not needed --> default _q config
    * -Q option for base quality threshold - not needed --> default _Q from config
    * --ff option for filter flags – not needed because of preprocessing??
    * --loption for samtools mpileup -l option – must elaborate..
    * --region option for restriction of regions on mpileup -l – must elaborate..
    * --debug – not needed

## Initiation

### imports

In [9]:
import os
from code import run

### snakemake config

In [10]:
config = {'EB':{'run': True}}
params = {}
params['map_quality'] = 20
params['base_quality'] = 15
params['filter_flags'] = 'UNMAP,SECONDARY,QCFAIL,DUP'
params['loption'] = True
config['EB']['threads'] = 1
config['EB']['params'] = params

### function args
will be passed during the function call

In [11]:
args = {}
args['mut_file'] = 'testdata/input.anno'
args['tumor_bam'] = 'testdata/tumor.bam'
args['pon_list'] = 'testdata/list_normal_sample.txt'
args['output_path'] = 'output/output.anno'
args['region'] = ''

### load the config and GLOBAL STATE into variable _

In [12]:
debug_mode = True
params = config['EB']['params']
threads = config['EB']['threads']
_q = str(params['map_quality'])  # mapping quality=20
_Q = params['base_quality']      # base quality=15
filter_quals = ''
for qual in range( 33, 33 + _Q ): 
    filter_quals += chr( qual )  # qual asciis for filtering out
_ff = params['filter_flags']     # 'UNMAP,SECONDARY,QCFAIL,DUP'
is_loption = params['loption']   # True
log_path = 'output/logs'         # log_file: 'output/logs/'
log_file = os.path.join(log_path, f"{os.path.splitext(os.path.basename(args['output_path']))[0]}.log")
# dump into _
_ = {'q':_q, 'Q':_Q, 'filter_quals': filter_quals, 'ff':_ff, 'log':log_file, 'threads':threads, 'debug_mode':debug_mode}

## Single threading

In [13]:
_['threads'] = 1
_['debug_mode'] = False
run.main(args,_)
!ls output

logs        output.anno


## Multithreading

In [14]:
_['threads'] = 3
_['debug_mode'] = True
run.main(args,_)

In [15]:
ls output/

logs/                             output.anno.sub.0.control.pileup
output.anno                       output.anno.sub.0.target.pileup
output.anno.0                     output.anno.sub.1
output.anno.0.region_list.bed     output.anno.sub.1.control.pileup
output.anno.1                     output.anno.sub.1.target.pileup
output.anno.1.region_list.bed     output.anno.sub.2
output.anno.2                     output.anno.sub.2.control.pileup
output.anno.2.region_list.bed     output.anno.sub.2.target.pileup
output.anno.sub.0


## using advanced dataframes for getting the pileup files 

### get anno and pileup as dataframes (too big?)

In [ ]:
anno = pd.read_csv('testdata/input.anno', sep='\t', header=None, names=['Chr','Start', 'End', 'ref', 'var'], dtype={'Chr': str, 'Start':int, 'End': int})
tumorpileup = pd.read_csv('output/tumor.pileup', sep='\t', header=None, names=['Chr', 'Start', 'ref', 'depth', 'reads', 'mapQ'], dtype={'Start':int, 'reads':str, 'mapQ': str})
pd.merge(anno,tumorpileup, on=['Chr', 'Start'])


In [ ]:
threads = 4
split = round(len(tumorpileup.index) / threads)
splits = np.array_split(tumorpileup, 4)

In [ ]:
splits[0]

In [1]:
import subprocess
from subprocess import PIPE
!ls testdata/

clean.pileup           normalreference3.bam   normalreference9.bai
input.anno             normalreference4.bai   normalreference9.bam
input.vcf.gz           normalreference4.bam   output.golden.anno
list_normal_sample.txt normalreference5.bai   output.golden.vcf
normalreference1.bai   normalreference5.bam   region.txt
normalreference1.bam   normalreference6.bai   testing.pileup
normalreference10.bai  normalreference6.bam   tumor.bai
normalreference10.bam  normalreference7.bai   tumor.bam
normalreference2.bai   normalreference7.bam   tumor.pileup
normalreference2.bam   normalreference8.bai   tumor_region.pileup
normalreference3.bai   normalreference8.bam


In [4]:
cmd1 = ['samtools', 'mpileup', '-l', 'testdata/region.txt', 'testdata/tumor.bam']
cmd2 = ['sed', '-E', 's/\^\]|\$//g']
with open('testdata/testing.pileup','w') as fileout:
    samtools = subprocess.Popen(cmd1, stdout=PIPE)
    sed = subprocess.Popen(cmd2, stdin=samtools.stdout, stdout=fileout)
    samtools.stdout.close()
    success = sed.communicate()

In [26]:
success

(b'chr11\t193067\tN\t40\tTTTTTTTTTTTTTTTTTTTTTTTTTtTTTtTtTTTTTtTT\tqqqIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII\nchr11\t193068\tN\t40\tTTTTTTTTTTTTTTTTTTTTTTTTtTTTttTtTTTTTtTT\tqqIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII\n',
 None)